# Plot and fit fluorescence polarization data for measuring direct binding or competitive inhibition
This script extracts data from an Excel spreadsheet exported from a BMG plate reader. Based on user input,
it calculates the ligand/inhibitor concentration for each data point, fits the data to the appropriate model, and
plots the results. The input data (in tabular format), plots, and fit results can be downloaded.

### Required input
Data from a fluorescence polarization experiment collected on a BMG plate reader (tested on a ClarioStar), exported
to an Excel spreadsheet (*.xlsx). 

### Optional input
A separate text file (*.asc or *.txt) which contains information about experimental conditions and the layout of the 
plate for each experiment. The format for this text file is detailed below and in the example "experiments.txt" file.

### Upload your data files to the local directory and run the following cell to initialize the script

Then select the data file and experiment list from the appropriate dropdown. The data file is the Excel spreadsheet, while the experiment 
list is the text file containing information about the experiments. You can choose "Default" for the experiment list, in which case you 
need to provide the number of experiments you set up. In the next step, a table will be generated based on this information, which can be 
edited as appropriate.

In addition, you must provide the starting row and column numbers for where the data is located in the Excel spreadsheet. The default
values are a typical location, but it depends on how the data was analyzed and saved by the BMG software. Open the spreadsheet in 
Excel and note the row number for the first line of data, the column containing the sample names (e.g., Sample X1, Sample X2...), and
the column containing the fluorescence polarization ratio. Do not choose the average values (if provided); this script calculates the
average and standard deviation for you. 

#### Important
If you upload your files to the local directory after initializing the script, you will need to re-run this cell so they appear in the
dropdown lists.

In [ ]:
# initialize script
import ipywidgets as widgets
import glob
import panel as pn
from functions import extract_data, setup_exps, fit_data, pltly, \
                        pltly_nofit, create_table, extract_data, \
                            combine_data, make_tables

pn.extension('tabulator')

row_start = pn.widgets.IntInput(name = "Start row:", value = 11, step =1 , start = 0, end = 1000, width=100)
num_exps = pn.widgets.IntInput(name = "Number of exps:", value = 1, step =1 , start = 1, end = 100, width=100)
sample_col = pn.widgets.Select(name = "Sample column:", value = 'C', options=['A','B','C','D','E','F','G','H','I','J'], width=100)
FP_col = pn.widgets.Select(name = "FP column:", value = 'F', options=['A','B','C','D','E','F','G','H','I','J'], width=100)

select_file = pn.widgets.Select(name='Select data file:', options = glob.glob("*.xlsx"))
select_list = pn.widgets.Select(name='Select experiment list:', options = ["Default"] + (glob.glob("*.asc") + glob.glob("*.txt")))
display(pn.Row(select_file, select_list), pn.Row(num_exps, row_start, sample_col, FP_col))



### After selecting files and entering the needed information, run the following cell

In [ ]:
# Set up experiments
fit_param_df = setup_exps(select_list.value, num_exps.value)

df_widget = pn.widgets.Tabulator(fit_param_df, 
                                 sortable=False,
                                 show_index = False,
                                 theme = 'bootstrap',
                                 layout = 'fit_data',
                                 header_align = 'center'
                                )


plot_fit = pn.widgets.Checkbox(value=True, name='Fit data?', disabled=False)

weight_err = pn.widgets.Checkbox(value=False, name='Error weighted fit?', disabled=False)

show_table = pn.widgets.Checkbox(value=True, name='Results table?', disabled=False)

display(df_widget, pn.Row(plot_fit, weight_err, show_table))

### Edit the experiment information
In the table above, provide a name for each **Experiment**, the number of serial dilutions collected (**points**) for that experiment, the **start**ing number (e.g., "1" if it starts at the first data point, "Sample X1"), the initial concentration (**[titrant]**) of the serially diluted component (ligand or inhibitor), **dilution** factor (e.g., "2.0" if you serially diluted 1:2), the fixed concentration (**[fluoro]**) of the fluoroscently labeled protein/ligand, the **type** of experiment ("bind" for direct binding and "inhib" for a competitive inhibition experiment), and the **units** for the input data.

If it is a competitive inhibitor experiment, provide the known binding constant (**Kd1**) for the interaction being disrupted (not for the inhibitor-protein interaction), and the fixed protein concentration (**Mt**) for the unlabeled protein. Otherwise, leave these set at 0.0.

### Choose options
Whether to fit the data, and if so, to weight the data based on the standard error, and whether to generate a results table.

### Run the following cell to perform the analysis and generate plots and tables


In [ ]:
# Fit and plot experiments
data_filename = select_file.value
skip_row = int(row_start.value - 1)
cols = str(sample_col.value) + ',' + str(FP_col.value) 

avg_data = extract_data(data_filename, skip_row, cols)
data = combine_data(avg_data, fit_param_df)

tables, tables_df = make_tables(data)

def save_to_csv(df):
    from io import StringIO
    df_sio = StringIO()
    df.to_csv(df_sio)
    df_sio.seek(0)
    return df_sio

def save_to_html(fig):
    from io import StringIO
    html_sio = StringIO()
    fig.write_html(html_sio)
    html_sio.seek(0)
    return html_sio

def save_tables_to_csv(df_list):
    from io import StringIO
    tables_sio = StringIO()
    i=0
    tables_sio.write(",FP data tables\n")
    for df in df_list:
        tables_sio.write("," + data[i][0] + "\n")
        df.to_csv(tables_sio)
        tables_sio.write("\n")
        i+=1
    tables_sio.seek(0)
    return tables_sio
    
def save_to_png(fig):
    from io import BytesIO
    png_bytes = fig.to_image(format="png", scale=10)
    png_bo = BytesIO(png_bytes)
    return png_bo


def save_to_pdf(fig):
    from io import BytesIO
    pdf_bytes = fig.to_image(format="pdf", scale=10)
    pdf_bo = BytesIO(pdf_bytes)
    return pdf_bo


pn.extension("plotly")
if plot_fit.value:
    out_params = fit_data(data, weight=weight_err.value)

if plot_fit.value:
  fig = pltly(data, out_params,
        num_of_col = 2,
        logX = True,
        bgcol='white',
        savefile = False,
        savename = data_filename[:-5] + '.png',
        savescale = 4,
        writehtml = False,
        htmlfilename = data_filename[:-5] + '.html')
else:
  fig = pltly_nofit(data,
        num_of_col = 2,
        logX = True,
        bgcol='white',
        savefile = False,
        savename = data_filename[:-5] + '.png',
        savescale = 4,
        writehtml = False,
        htmlfilename = data_filename[:-5] + '.html')

plotly_pane = pn.pane.Plotly(fig)

if show_table.value:
    if not plot_fit.value:
        out_params = fit_data(data, weight=weight_err.value)
    results_table_df = create_table(data,out_params)
    results_table = pn.widgets.Tabulator(results_table_df, 
                                        sortable=False, 
                                        show_index=False, 
                                        disabled=True,
                                        theme='bootstrap',
                                        header_align = 'center')
    # results_filename, results_download_button = results_table.download_menu(
    #                                                 text_kwargs={'name': 'Enter filename', 'value': 'results.csv'},
    #                                                 button_kwargs={'name': 'Download table'}
                                                                            # )
    save_csv = pn.widgets.FileDownload(
                callback=pn.bind(save_to_csv, results_table_df), filename='results.csv'
            )
save_tables = pn.widgets.FileDownload(
            callback=pn.bind(save_tables_to_csv, tables_df), filename='tables.csv'
            )

save_file = pn.widgets.TextInput(name='Filename',placeholder='Enter file name for plots', width = 200)
save_png = pn.widgets.FileDownload(
                callback=pn.bind(save_to_png, fig), filename='plot.png'
            )
save_pdf = pn.widgets.FileDownload(
                callback=pn.bind(save_to_pdf, fig), filename='plot.pdf'
            )

save_html = pn.widgets.FileDownload(
                callback=pn.bind(save_to_html, fig), filename='plot.html'
                )

if show_table.value:
    display(pn.Row(pn.Column(plotly_pane, pn.Row(save_png, save_pdf, save_html))), pn.Column(tables, save_tables, results_table, save_csv))
else:
    display(pn.Row(pn.Column(plotly_pane, pn.Row(save_png, save_pdf, save_html), pn.Column(tables, save_tables))))